## Carnd-term1

### Project 4. Advanced Lane Finding

**Advanced Lane Finding Project**

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

[//]: # (Image References)

[image1]: ./examples/undistort_output1.png "Undistorted"
[image2]: ./test_images/test1.png "Road Transformed"
[distortion-corrected-image]: ./writeup_images/distortion-corrected-image.png "distortion-corrected"
[channel_checks]: ./writeup_images/channel_checks.png "Channel Checks png"
[gray_filter]: ./writeup_images/gray_filter.png "gray_filter"
[abs_sobel_filter]: ./writeup_images/abs_sobel_filter.png "abs_sobel_filter"
[mag_sobel_filter]: ./writeup_images/mag_filter.png "mag_sobel_filter"
[dir_sobel_filter]: ./writeup_images/dir_filter.png "dir_sobel_filter"
[combine_sobel_filter]: ./writeup_images/combine_filter.png "combine_filter"

[roi]: ./writeup_images/roi.png "roi"
[roi_pers_trans]: ./writeup_images/roi_pers_trans.png "roi_pers_trans"
[combine_transform_test]: ./writeup_images/combine_transform_test.png "combine_transform_test"

[his_vis]: ./writeup_images/histgram_vis.png "his_vis"
[window_sliding_vis]: ./writeup_images/window_sliding_vis.png "window_sliding_vis"
[fit_base_on_old_frame]: ./writeup_images/fit_base_on_old_frame.png "fit_base_on_old_frame"
[map_lane]: ./writeup_images/map_lane.png "map_lane"


[image4]: ./examples/warped_straight_lines.jpg "Warp Example"
[image5]: ./examples/color_fit_lines.jpg "Fit Visual"
[image6]: ./examples/example_output.jpg "Output"
[video1]: ./project_video.mp4 "Video"

[image2_undistorted]: ./test_images/test1_undistorted.jpg "Road Transformed"


## [Rubric](https://review.udacity.com/#!/rubrics/571/view) Points

### Here I will consider the rubric points individually and describe how I addressed each point in my implementation.  

### Camera Calibration

#### 1. Briefly state how you computed the camera matrix and distortion coefficients. Provide an example of a distortion corrected calibration image.

The code for this step is contained in the first code cell of the IPython notebook located in "./examples/example.ipynb".  

I start by preparing "object points", which will be the (x, y, 0) coordinates of the chessboard corners in the world. Here I am assuming the chessboard is fixed on a plane with z=0 in the real world's 3D coordinates systems, such that the object points are the same for each calibration image.  Obviously, in the real world, (x,y) should lies on a meshgrid with equal intervals. Thus, `objp` is just a replicated array of coordinates, and `objpoints` will be appended with a copy of it every time I successfully detect all chessboard corners in a test image.  `imgpoints` will be appended with the (x, y) pixel position of each of the corners in the image plane with each successful chessboard detection.  

I then used the output `objpoints` and `imgpoints` to compute the camera calibration and distortion coefficients using the `cv2.calibrateCamera()` function.  I applied this distortion correction to the test image using the `cv2.undistort()` function and obtained this result: 

![alt text][image1]

### Pipeline (single images)

#### 1. Provide an example of a distortion-corrected image.

To demonstrate this step, I apply the distortion correction to one of the test images like this one:
![alt text][distortion-corrected-image]
From the bottom of the two pics, we can feel the difference.

#### 2. Describe how (and identify where in your code) you used color transforms, gradients or other methods to create a thresholded binary image.  Provide an example of a binary image result.

To get a nearly perfert binary image, a well chosen channel of the image play an important role. So I firstly choose the best channel by visualizing the **'Channel Checks'** firstly.

![alt text][channel_checks]

From above pictures, we can see that 1) LAB B-channel has clear edge at yellow line; 2) HSL L-channel has clear edge at white line mask. 3) HLS S-channel has clear edge at both yellow line and white line, this is why udacity use this S-channel.

Hence, I follow udacity tutorial:
- **Step1:** Filtered by gray threshold on S-channel.
- **Step2:** Filtered by one kind of Sobel gradients i.e., one of abs, magnitude or direction.
- **Step3:** Combine with above two results. ** WHY combination?** Because sometimes such as light condition etc, only with **Step1** or **Step2** singly will fail to dectect lines. By combination, we make the detection **More Robust!**

### Step1 (./exmaples/example.ipynb In[8]):
```python
def color_binary(img, s_threshold=(100,255) ):
    
    hls_image= cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls_image[:,:,2]
    
    color_binary_img = np.zeros_like(s_channel)
    color_binary_img[(s_channel >= s_threshold[0]) & (s_channel <= s_threshold[1])] = 1  

    return color_binary_img
```
The effect is:
![alt text][gray_filter.png]

### Step2 (./exmaples/example.ipynb In[10-]):
In the case of lane lines, we're interested only in edges of a particular (vertical) orientation. So following from udacity tutorial, I firstly visualize the effect of **abs threshold of x or y orientation Sobel**. From **'Channel Checks'**, we can see than take the Gray Image for getting Gradient Image is **NOT a GOOD idea** (./exmaples/example.ipynb In[10]), because the yellow line nearly disappears. So we should take a robust channel as selection. Here I select the RGB B-channel which is a **good selection** (./exmaples/example.ipynb In[11]).

2.1 Then I use abs_sobel_thresh to filter (./exmaples/example.ipynb In[12]). The result is below:
![alt text][abs_sobel_filter]

2.2 Use mag_sobel_thresh to filter (./exmaples/example.ipynb In[13]). The result is below:
![alt text][mag_sobel_filter]

2.3 Use mag_sobel_thresh to filter (./exmaples/example.ipynb In[14]). The result is below:
![alt text][dir_sobel_filter]

From above three filters, I would rather select Magnitude Sobel. Because it has sharpened effect on lines. I think **combine Magnitude Sobel with Color Space** would give a good result for line dectection. Hence next I do combination.

2.4 Combination filter (./exmaples/example.ipynb In[15]). The result is below:
![alt text][combine_sobel_filter]



#### 3. Describe how (and identify where in your code) you performed a perspective transform and provide an example of a transformed image.

The code for my perspective transform includes a function called `transform()`, which appears in lines 17 the file `example.ipynb` (./examples/example.ipynb).  The `transform()` function takes as inputs an image (`img`), as well as source (`src`) and destination (`dst`) points.  I chose the hardcode the source and destination points in the following manner:

```python
src = np.float32([(580,450),
                  (720,450), 
                  (240,680), 
                  (1160,680)])
dst = np.float32([(130,0),
                  (w-130,0),
                  (130,h),
                  (w-130,h)])
```

I verified that my perspective transform was working as expected by drawing the `src` and `dst` points onto a test image and its warped counterpart to verify that the lines appear parallel in the warped image.

Before perspective transformed, it looks:
![alt text][roi]
After perspective transformed, it looks:
![alt text][roi_pers_trans]

Then I wrap filter and transform in to a pipeline function as bellows:
```python
def pipeline(img):
    # Undistort
    img_undistort = undistort(img)
    # Perspective Transform
    img_trans, M, Minv = transform(img_undistort, src, dst)
    # Combined Sobel
    combined_img = combine_sobel(img_trans, kernel=11, mag_th=(30, 255), s_th=(140,255))
    return combined_img, Minv 
```
With it I test Combined Sobel and Transform on all test images (./exmaples/example.ipynb In[54]):
![alt text][combine_transform_test]

#### 4. Describe how (and identify where in your code) you identified lane-line pixels and fit their positions with a polynomial?

Line Finding Method: Peaks in a Histogram. I first take a histogram along all the columns in the lower half of the image like this:
![alt text][his_vis]

Then I define a method to fit polynomial to binary image with lines extracted, using sliding window (./exmaples/example.ipynb In[21])

The visualization is:
![alt text][window_sliding_vis]

If the sequential image frame will not change significantly, we can fit the future frame based on the result of the last fit, as so to save computing resource significantly (./exmaples/example.ipynb In[23]).

The visualization of fitting based on old frame is:
![alt text][fit_base_on_old_frame]
**Good Job!**
1. The blue shadow polygon is means the search window area educated from the old fitted frame;
2. The fitted yellow line is the new fitting for the new image. We can see that it fits relatively satisfying.


#### 5. Describe how (and identify where in your code) you calculated the radius of curvature of the lane and the position of the vehicle with respect to center.

For all details, I did this in (./exmaples/example.ipynb In[31])

#### 6. Provide an example image of your result plotted back down onto the road such that the lane area is identified clearly.

I implemented this step in lines # through # in my code in `yet_another_file.py` in the function `map_lane()`.  Here is an example of my result on a test image:

![alt text][map_lane]

### Pipeline (video)

#### 1. Provide a link to your final video output.  Your pipeline should perform reasonably well on the entire project video (wobbly lines are ok but no catastrophic failures that would cause the car to drive off the road!).

Here's a [link to my video result](./project_video_output.mp4)

### Discussion

#### 1. Briefly discuss any problems / issues you faced in your implementation of this project.  Where will your pipeline likely fail?  What could you do to make it more robust?
A likely reason to lead to fail is that the result of combination Sobel is not very good. To address this, I should find more robust channel, filter method or even the other image reprensentation.